## NLP 작업 종류 

* NER(Named Entity Recognition) : 문장 내의 각 단어를 `사람` `장소` `동물` 등의 정보를 분류하는 작업
* POS (Part of Speech) Tag : 단어의 품사를 맞추는 작업
* Language Model : 생성할 문장의 각 단어에 대한 다음 단어를 예측

## 개발 작업 순서

### 데이터
목적에 필요한 정답이 포함된 데이터를 수집하거나 직접 생성 합니다.
이 프로젝트의 경우는 각 단어가 명사인지 여부를 표시해놓은 데이터가 필요합니다.

### Vocabulary 생성
수집된 데이터를 일정한 단위로 일련번호를 부여하는 과정입니다.
자연어처리에서는 컴퓨터에 입력할 때 글자가 아닌 vocabulary에 부여된 일련번호를 입력합니다.

### 학습 및 평가용 데이터 생성
데이터를 vocabulary를 이용해서 일련번호로 변경 후 학습에 적합한 형태로 가공하는 과정입니다.

### 모델링
목적에 적합한 모델을 생성합니다.

### 학습
데이터를 이용해서 모델을 학습하는 과정입니다.

### 테스트
학습된 모델을 테스트하는 과정입니다.
모델을 테스트해서 성능이 기준치보다 좋으면 서비스에 배포 합니다.
만일 모델의 성능이 기준치보다 성능이 낮은 경우는 원인을 분석하고 문제가 된다고 생각되는 과정으로 돌아가서 그 과정을 개선하고 다시 학습하고 테스트해 봅니다.

### 배포
학습된 모델을 이용하여 실제 문제에 적용해서 문제를 개선하는 과정입니다.

In [ ]:
# 개발환경
import argparse
import random
import matplotlib.pyplot as plt
import numpy as np
import torch

#### 추가 정보

* 파이썬에서 `*` 는 튜플에서, `**` 는 딕셔너리에서 iterator 작업이 가능하게끔 만들어준다.

In [ ]:
# Config 파일 설정
args = {
    # random seed value
    "seed": 1234,
    # number of epoch
    "n_epoch": 200,
    # number of batch
    "n_batch": 2,
    # learning rate
    "lr": 0.001,
    # weights 저장 위치
    "save_path": "01-01-sequence-prediction.pth",
    # CPU 또는 GPU 사용여부 결정
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}
args = argparse.Namespace(**args)

print(args)

In [ ]:
# random seed 설정
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

In [ ]:
# input text
raw_inputs = [
    "나는 학생 입니다",
    "나는 좋은 선생님 입니다",
    "당신은 매우 좋은 선생님 입니다"
]

In [ ]:
# 정답: 명사(1), 기타(0)
raw_labels = [
    [0, 1, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1, 0]
]

In [ ]:
words = []

for s in raw_inputs:
    words.extend(s.split())

print(words)

In [ ]:
# 중복제거

words = list(dict.fromkeys(words))

print(words)

In [ ]:
# 각 단어별 일련번호
word_to_id = {"[PAD]": 0, "[UNK]": 1}
for w in words:
    word_to_id[w] = len(word_to_id)

print(word_to_id)

In [ ]:
# 각 번호별 단어
id_to_word = {i: w for w, i in word_to_id.items()}

print(id_to_word)

In [ ]:
# 입력 데이터
inputs = []
for s in raw_inputs:
    inputs.append([word_to_id[w] for w in s.split()])
    
print(inputs)

In [ ]:
# 정답 데이터
labels = raw_labels

print(labels)

In [ ]:
class SimpleDataSet(torch.utils.data.Dataset):
    """데이터 셋 클래스"""
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        assert len(self.inputs) == len(self.labels)
        return len(self.labels)
    
    def __getitem__(self, index):
        return (
            torch.tensor(self.inputs[index]),
            torch.tensor(self.labels[index])
        )

    def collate_fn(self, batch):
        inputs, labels = list(zip(*batch))

        inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first = True, padding_value = 0)
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first = True, padding_value = 0)

        return [
                 inputs,
                 labels
        ]

In [ ]:
dataset = SimpleDataSet(inputs, labels)

print(len(dataset))
print(dataset[1])

In [ ]:
# random sample data
sampler = torch.utils.data.RandomSampler(dataset)

In [ ]:
# train loader
train_loader = torch.utils.data.DataLoader(dataset, batch_size=args.n_batch, sampler=sampler, collate_fn=dataset.collate_fn)

In [ ]:
# dataset
dataset = SimpleDataSet(inputs, labels)
# valid loader
valid_loader = torch.utils.data.DataLoader(dataset, batch_size=args.n_batch, sampler=None, collate_fn=dataset.collate_fn)

In [ ]:
# model tutorial

t_inputs = torch.tensor(inputs[:1])
t_labels = torch.tensor(labels[:1])

print(t_inputs, t_labels)

In [ ]:
# 단어를 vector로 변환시킨다
embed = torch.nn.Embedding(len(word_to_id), 4)
hidden = embed(t_inputs)

print(len(word_to_id))
print(t_inputs.shape)
print(hidden.shape, hidden)

In [ ]:
# 단어 명사(1) 또는 기타(0) 예측
linear = torch.nn.Linear(4, 2)
logits = linear(hidden)

print(logits.shape, logits)

In [ ]:
print(logits.shape)
print(logits.view(-1, logits.size(-1)))
print(t_labels.shape)
print(t_labels.view(-1).shape)

In [ ]:
# CrossEntropy loss 
loss_fn = torch.nn.CrossEntropyLoss()
loss = loss_fn(logits.view(-1, logits.size(-1)), t_labels.view(-1))

In [ ]:
print(loss)

In [ ]:
class SequencePrediction(torch.nn.Module):
    def __init__(self, n_vocab):
        super().__init__()
        self.embed = torch.nn.Embedding(n_vocab, 4)
        self.linear = torch.nn.Linear(4, 2)

    def forward(self, inputs):
        hidden = self.embed(inputs)
        logits = self.linear(hidden)
        return logits

In [ ]:
# 학습용 모델 생성
model = SequencePrediction(len(word_to_id))
model.to(args.device)

print(model)

In [ ]:
# loss & optimizer 생성
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

In [ ]:
def accuracy_fn(logits, labels):
    _, indices = logits.max(-1)
    matchs = torch.eq(indices, labels).cpu().numpy()
    total = np.ones_like(matchs)
    return np.sum(matchs) / max(1, np.sum(total))

In [ ]:
def train_epoch(args, model, loader, loss_fn, optimizer):
    """
    1 epoch 학습
    :param args: 입력 arguments
    :param model: 모델
    :param loader: 데이터로더
    :param loss_fn: loss 계산함수
    :param optimizer: optimizer
    """
    # model을 train 모드로 전환
    model.train()
    # loss 및 accuracy 저장
    losses, access = [], []
    # data loader에서 batch단위로 처리
    for batch in loader:
        # optimizer 초기화
        optimizer.zero_grad()
        # batch 입력값 처리 (CPU or GPU)
        inputs, labels = map(lambda v: v.to(args.device), batch)
        # 모델 실행
        logits = model(inputs)
        # loss 계산
        loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
        loss.backward()
        # model weight 변경
        optimizer.step()
        # loss 저장
        loss_val = loss.item()
        losses.append(loss_val)
        # accuracy 계산 및 저장
        acc_val = accuracy_fn(logits, labels)
        access.append(acc_val)

    return np.mean(losses), np.mean(access)

In [ ]:
def eval_epoch(args, model, loader, loss_fn):
    """
    1 epoch 평가
    :param args: 입력 arguments
    :param model: 모델
    :param loader: 데이터로더
    :param loss_fn: loss 계산함수
    """
    # model을 eval 모드로 전환
    model.eval()
    # loss 및 accuracy 저장
    losses, access = [], []
    # 실행시에 gradint 계산 비활성화
    with torch.no_grad():
        for batch in loader:
            # batch 입력값 처리 (CPU or GPU)
            inputs, labels = map(lambda v: v.to(args.device), batch)
            # 모델 실행
            logits = model(inputs)
            # loss 계산
            loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
            # loss 저장
            loss_val = loss.item()
            losses.append(loss_val)
            # accuracy 계산 및 저장
            acc_val = accuracy_fn(logits, labels)
            access.append(acc_val)

    return np.mean(losses), np.mean(access)

In [ ]:
# 학습 history
history = {"train_loss": [], "train_acc": [], "valid_loss": [], "valid_acc": []}
# 가장 좋은 acc 값
best_acc = 0

In [ ]:
# 학습 및 평가
for e in range(args.n_epoch):
    train_loss, train_acc = train_epoch(args, model, train_loader, loss_fn, optimizer)
    valid_loss, valid_acc = eval_epoch(args, model, valid_loader, loss_fn)
    # 학습 history 저장
    history["train_loss"].append(train_loss)
    history["train_acc"].append(train_acc)
    history["valid_loss"].append(valid_loss)
    history["valid_acc"].append(valid_acc)
    # 학습과정 출력
    print(f"eopch: {e + 1:3d}, train_loss: {train_loss:.5f}, train_acc: {train_acc: .5f}, valid_loss: {valid_loss:.5f}, valid_acc: {valid_acc:.5f}")
    # best weight 저장
    if best_acc < valid_acc:
        best_acc = valid_acc
        # 저장
        torch.save(
            {"state_dict": model.state_dict(), "valid_acc": valid_acc},
            args.save_path,
        )
        # 저장내용 출력
        print(f"  >> save weights: {args.save_path}")

In [ ]:
def draw_history(history):
    """
    학습과정 그래프 출력
    :param history: 학습 이력
    """
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history["train_loss"], "b-", label="train_loss")
    plt.plot(history["valid_loss"], "r--", label="valid_loss")
    plt.xlabel("Epoch")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history["train_acc"], "k--", label="train_acc")
    plt.plot(history["valid_acc"], "k--", label="valid_acc")
    plt.xlabel("Epoch")
    plt.legend()

    plt.show()

In [ ]:
draw_history(history)


In [ ]:
# 배포

# 배포용 모델 생성
model = SequencePrediction(len(word_to_id))
model.to(args.device)

# 저장된 데이터 로드
save_dict = torch.load(args.save_path)

# 학습된 weights로 모델 초기화
model.load_state_dict(save_dict['state_dict'])

In [ ]:
def do_predict(word_to_id, model, string):
    """
    입력에 대한 답변 생성하는 함수
    :param word_to_id: vocabulary
    :param model: model
    :param string: 입력 문자열
    """
    # token 생성
    token = [word_to_id[w] for w in string.strip().split()]

    model.eval()
    with torch.no_grad():
        inputs = torch.tensor([token]).to(args.device)
        logits = model(inputs)
        _, indices = logits.max(-1)
        y_pred = indices[0].numpy()
    result = ["명사" if i == 1 else "기타" for i in y_pred]
    return result

In [ ]:
# 예측 실행
do_predict(word_to_id, model, "당신은 선생님 입니다")